# <font color = 'color'>[R텍마05] 2부2장3장_텍스트분석을 위한 정규표현식2</font>

### 필요 패키지 인스톨 및 라이브러리 로딩

In [1]:
# 패키지 설치&로딩 사용자정의함수 작성
mypak <- function(pkg){
    # 필요패키지 설지여부를 체크해 미설치패키지 목록을 저장
    new.pkg <- pkg[!(pkg %in% rownames(installed.packages()))]
    
    # 미설치 패키지를 일괄 인스톨 실시
    if (length(new.pkg)) 
        install.packages(new.pkg, dependencies = TRUE)
    
    # 필요패키지를 일괄 로딩실시
    sapply(pkg, require, character.only = TRUE)  
}

In [2]:
# 패키지 설치&로딩 사용자정의함수 이용
pak <- c('stringr', 'magrittr', 'purrr')
# - stringr: 문자열 분석패키지
# - magrittr: 파이프연산자 %>%
# - purrr: 데이터순환연산(벡터리사이클링, 벡터브로드캐스팅, 벡터연산) map시리즈 패키지

mypak(pak)

Loading required package: stringr
Loading required package: magrittr
Loading required package: purrr

Attaching package: 'purrr'

The following object is masked from 'package:magrittr':

    set_names

The following object is masked from 'package:jsonlite':

    flatten



stringr magrittr    purrr 
    TRUE     TRUE     TRUE

# <font color = 'blue'>텍스트셋에서 특정패턴 탐색: 대상객체가 1개 요소인 경우(p.59, p.73)</font>
* 원본 텍스트셋의 내용과 구조에 상관없이 1개 문자요소로 간주하여 분석
* base패키지(p.59)와 stringr패키지(p.73) 병행사용
* 텍스트셋 안에 특정패턴의 포함여부, 출현위치, 출현내용, 출현횟수 파악
* 텍스트셋 안에 원본패턴을 목표패턴으로 변경

## 텍스트 데이터 준비

In [3]:
# 분석대상 텍스트 데이터 준비
my <- 'The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. 
The "Task Views" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for the collaborative development of R packages, R-related software, and projects. R-Forge also hosts many unpublished beta packages, and development versions of CRAN packages.'

sprintf('검색대상 객체내용:')
print(my)

sprintf('검색대상 객체유형: %s객체', class(my))
sprintf('검색대상 객체길이: %d요소', length(my))

sprintf('검색대상 객체글자수: %d개', nchar(my))
# - sprintf('검색대상 객체글자수: %d글자', str_length(my))

[1] "검색대상 객체내용:"

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform fo

[1] "검색대상 객체유형: character객체"

[1] "검색대상 객체길이: 1요소"

[1] "검색대상 객체글자수: 1166개"

## 텍스트셋에 특정패턴 포함여부 파악
* base:: 논리값으로 grep(), 요소인덱스번호로 grepl()
* stringr:: 논리값으로 str_detect(), 요소인덱스번호로 str_detect() & which()

In [4]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 포함여부를 논리값으로

In [5]:
# 텍스트셋 안에 특정패턴 포함여부를 논리값으로 도출 
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 들어 있으면 TRUE, 없으면 FALSE 출력

sprintf('base::grepl() 함수이용 특정패턴 포함유무')
print(grepl(pattern = keyword, x = my))
print(grepl(pattern = keyword, x = my, ignore.case = TRUE)) # 대소문자 무시옵션

sprintf('stringr::str_detect() 함수이용 특정패턴 포함유무')
str_detect(pattern = keyword, string = my) %>% print
str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my) %>% print  # 대소문자 무시옵션

[1] "base::grepl() 함수이용 특정패턴 포함유무"

[1] FALSE
[1] TRUE


[1] "stringr::str_detect() 함수이용 특정패턴 포함유무"

[1] FALSE
[1] TRUE


### 포함여부를 요소인덱스번호로

In [6]:
# 텍스트셋 안에 특정패턴 포함여부를 요소인덱스번호로 도출 
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 있으면 정수숫자 1을 출력해 주며, 
#   만일 해당하는 텍스트요소가 없으면, NULL의미를 가진 integer(0) 문구가 나옴

sprintf('base::grep() 함수이용 특정패턴 포함인덱스')
print(grep(pattern = keyword, x = my))
print(grep(pattern = keyword, x = my, ignore.case = TRUE)) # 대소문자 무시옵션

sprintf('stringr::str_detect() & base::which() 함수이용 특정패턴 포함인덱스')
str_detect(pattern = keyword, string = my) %>% which %>% print
str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my) %>% which %>% print  # 대소문자 무시옵션

[1] "base::grep() 함수이용 특정패턴 포함인덱스"

integer(0)
[1] 1


[1] "stringr::str_detect() & base::which() 함수이용 특정패턴 포함인덱스"

integer(0)
[1] 1


### 포함여부를 요소내용으로

In [7]:
# 텍스트셋 안에 특정패턴 포함여부를 해당요소 내용으로 도출 
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 있으면 해당 요소의 내용을 출력해 주며, 
#   만일 해당하는 텍스트요소가 없으면, NULL의미를 가진 character(0) 문구가 나옴

#### base::grep() 함수이용

In [8]:
sprintf('base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE')
print(grep(pattern = keyword, x = my, value = TRUE))

cat('\n')

sprintf('base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE')
print(grep(pattern = keyword, x = my, ignore.case = TRUE, value = TRUE))

[1] "base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE"

character(0)



[1] "base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE"

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform fo

#### stringr::str_detect() 함수이용

In [9]:
sprintf('stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE')
my_det <- str_detect(pattern = keyword, string = my) 
my_det %>% print
my[my_det] %>% print

cat('\n')

sprintf('stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE')
my_det <- str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my) 
my_det %>% print
my[my_det] %>% print

[1] "stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE"

[1] FALSE
character(0)



[1] "stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE"

[1] TRUE
[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central pl

## 텍스트셋에 특정패턴 출현위치 파악
* base:: 최초출현위치 regexpr(), 모든출현위치 gregexpr(), 중첩검색패턴 최초출현위치 regexec()
* stringr:: 최초출현위치 str_locate(), 모든출현위치 str_locate_all()

In [10]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현위치

In [11]:
# 텍스트셋 안에 특정패턴 최초 출현위치 파악 
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 출현하는 최초 위치(자리)도 1개만 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, stringr::str_locate()은 NA를 출력함

sprintf('base::regexpr() 함수이용 최초 출현위치')
print(regexpr(pattern = keyword, text = my))
cat('\n')
print(regexpr(pattern = keyword, text = my, ignore.case = TRUE)) # 대소문자 무시옵션

sprintf('stringr::str_locate() 함수이용 최초 출현위치')
print(str_locate(pattern = keyword, string = my))
cat('\n')
print(str_locate(regex(pattern = keyword, ignore_case = TRUE), string = my)) # 대소문자 무시옵션

[1] "base::regexpr() 함수이용 최초 출현위치"

[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[1] 57
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE


[1] "stringr::str_locate() 함수이용 최초 출현위치"

     start end
[1,]    NA  NA

     start end
[1,]    57  64


### 모든 출현위치

In [12]:
# 텍스트셋 안에 특정패턴 모든 출현위치 파악 
# - 현재 1개 텍스트요소에 대한 검색인데, 검색패턴이 출현하는 모든 위치(자리)들을 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::gregexpr()은 -1을, stringr::str_locate_all()은 NA를 출력함

sprintf('base::gregexpr() 함수이용 모든 출현위치')
print(gregexpr(pattern = keyword, text = my))
print(gregexpr(pattern = keyword, text = my, ignore.case = TRUE)) # 대소문자 무시옵션

sprintf('stringr::str_locate_all() 함수이용 모든 출현위치')
print(str_locate_all(pattern = keyword, string = my))
print(str_locate_all(regex(pattern = keyword, ignore_case = TRUE), string = my)) # 대소문자 무시옵션

[1] "base::gregexpr() 함수이용 모든 출현위치"

[[1]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[1]]
[1]   57  206  732  949 1030 1115 1158
attr(,"match.length")
[1] 8 8 8 8 8 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



[1] "stringr::str_locate_all() 함수이용 모든 출현위치"

[[1]]
     start end

[[1]]
     start  end
[1,]    57   64
[2,]   206  213
[3,]   732  739
[4,]   949  956
[5,]  1030 1037
[6,]  1115 1122
[7,]  1158 1165



## 텍스트셋에 특정패턴 출현내용 파악
* base:: 최초출현내용 regexpr() & regmatches(), 모든출현내용 regexpr() & regmatches()  
* stringr:: 최초출현내용 str_extract(), 모든출현내용 str_extract_all()

In [13]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현내용

In [14]:
# 텍스트셋 안에 특정패턴 최초 출현내용 파악 

sprintf('base::regexpr() & regmatches 함수이용 최초 출현내용: 세부연산과정 상세')
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, regmatches는 character(0)을 출력 
# - 검색패턴이 해당 텍스트요소에 들어 있으면 
#   base::gregexpr()은 최초 출현위치 인덱스를, regmatches는 최초 출현내용을 출력 

# 검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우
tmp_loc <- regexpr(pattern = keyword, text = my, ignore.case = FALSE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my, m = tmp_loc)
print(tmp_ptr)

cat('\n#--------------------\n\n')

# 검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우
tmp_loc <- regexpr(pattern = keyword, text = my, ignore.case = TRUE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my, m = tmp_loc)
print(tmp_ptr)

[1] "base::regexpr() & regmatches 함수이용 최초 출현내용: 세부연산과정 상세"

[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

character(0)

#--------------------

[1] 57
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[1] "packages"


In [15]:
# 텍스트셋 안에 특정패턴 최초 출현내용 파악
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 출현하는 모든 내용도 1개만 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, regmatches는 character(0), 
#   stringr::str_extract()은 NA를 출력함

sprintf('base::regexpr() & regmatches  함수이용 최초 출현내용: 파이프연산자 이용')
regexpr(pattern = keyword, text = my) %>% regmatches(x = my) %>% print
regexpr(pattern = keyword, text = my, ignore.case = TRUE) %>% regmatches(x = my) %>% print

sprintf('stringr::str_extract() 함수이용 최초 출현내용')
str_extract(pattern = keyword, string = my) %>% print
str_extract(regex(pattern = keyword, ignore_case = TRUE), string = my) %>% print

[1] "base::regexpr() & regmatches  함수이용 최초 출현내용: 파이프연산자 이용"

character(0)
[1] "packages"


[1] "stringr::str_extract() 함수이용 최초 출현내용"

[1] NA
[1] "packages"


### 모든 출현내용

In [16]:
# 텍스트셋 안에 특정패턴 모든 출현내용 파악 

sprintf('base::gregexpr() & regmatches 함수이용 모든 출현내용: 세부연산과정 상세')
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::gregexpr()은 -1을, regmatches는 character(0)을 출력 
# - 검색패턴이 해당 텍스트요소에 들어 있으면 
#   base::gregexpr()은 모든 출현위치 인덱스를, regmatches는 모든 출현내용을 출력 

# 검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우
tmp_loc <- gregexpr(pattern = keyword, text = my, ignore.case = FALSE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my, m = tmp_loc)
print(tmp_ptr)

cat('\n#--------------------\n\n')

# 검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우
tmp_loc <- gregexpr(pattern = keyword, text = my, ignore.case = TRUE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my, m = tmp_loc)
print(tmp_ptr)

[1] "base::gregexpr() & regmatches 함수이용 모든 출현내용: 세부연산과정 상세"

[[1]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE


[[1]]
character(0)


#--------------------

[[1]]
[1]   57  206  732  949 1030 1115 1158
attr(,"match.length")
[1] 8 8 8 8 8 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE


[[1]]
[1] "packages" "packages" "packages" "packages" "packages" "packages" "packages"



In [17]:
# 텍스트셋 안에 특정패턴 모든 출현내용 파악
# - 현재 1개 텍스트요소에 대한 검색인데, 검색패턴이 출현하는 모든 내용을 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함

sprintf('base::gregexpr() & regmatches  함수이용 모든 출현내용: 파이프연산자 이용')
gregexpr(pattern = keyword, text = my) %>% regmatches(x = my) %>% print
gregexpr(pattern = keyword, text = my, ignore.case = TRUE) %>% regmatches(x = my) %>% print

sprintf('stringr::str_extract_all() 함수이용 모든 출현내용')
str_extract_all(pattern = keyword, string = my) %>% print
str_extract_all(regex(pattern = keyword, ignore_case = TRUE), string = my) %>% print

[1] "base::gregexpr() & regmatches  함수이용 모든 출현내용: 파이프연산자 이용"

[[1]]
character(0)

[[1]]
[1] "packages" "packages" "packages" "packages" "packages" "packages" "packages"



[1] "stringr::str_extract_all() 함수이용 모든 출현내용"

[[1]]
character(0)

[[1]]
[1] "packages" "packages" "packages" "packages" "packages" "packages" "packages"



## 테스트셋에 특정패턴 출현횟수 파악
* base:: gregexpr() & & regmatches & lenghts
* stringr:: str_count()

In [18]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 출현횟수: 세부연산과정 상세

In [19]:
# 대상텍스트에 변경대상 키워드 출현횟수를 파악
sprintf('base::gregexpr() & regmatches & lengths  함수이용 출현횟수: 세부연산과정 상세')

tmp_loc <- gregexpr(pattern = keyword, text = my, ignore.case = TRUE)
print(tmp_loc)
# - 해당표현이 출현한 모든 위치를 파악

tmp_ptr <- regmatches(x = my, m = tmp_loc)
print(tmp_ptr)
# - 해당표현이 출현한 모든 내용을 파악: 리스트객체임

lengths(tmp_ptr) %>% print
# - 리스트객체에 대한 리스트항목별 세부요소갯수 산정

[1] "base::gregexpr() & regmatches & lengths  함수이용 출현횟수: 세부연산과정 상세"

[[1]]
[1]   57  206  732  949 1030 1115 1158
attr(,"match.length")
[1] 8 8 8 8 8 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[1]]
[1] "packages" "packages" "packages" "packages" "packages" "packages" "packages"

[1] 7


### 출현횟수: 파이프연산자 이용

In [20]:
# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('base::gregexpr() & regmatches  함수이용 출현횟수: 파이프연산자 이용')
gregexpr(pattern = keyword, text = my, ignore.case = TRUE) %>% regmatches(x = my) %>% lengths %>% print

sprintf('stringr::str_count() 함수이용 출현횟수')
str_count(regex(pattern = keyword, ignore_case = TRUE), string = my) %>% print

[1] "base::gregexpr() & regmatches  함수이용 출현횟수: 파이프연산자 이용"

[1] 7


[1] "stringr::str_count() 함수이용 출현횟수"

[1] 7


## 원본패턴을 목표패턴으로 변경
* base:: 최초패턴을 변경 sub(), 모든패턴을 변경 gsub()
* stringr:: 최초패턴을 변경 str_replace(), 모든패턴을 변경 str_replace_all()

In [21]:
# 대상텍스트데이터 재확인
print(my)

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform fo

In [22]:
# 변경용 키워드 준비
src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴
# - 검색용 원본패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현내용 변경

In [23]:
# 텍스트데이터내 원본패턴 중 최초 출현내용을 목표패턴으로 변경

sprintf('base::sub() 함수이용 최초 출현내용 변경')
my_sub <- sub(pattern = src, replacement = tgt, x = my, ignore.case = TRUE)
print(my_sub)

sprintf('stringr::str_replace() 함수이용 최초 출현내용 변경')
my_rep <- str_replace(regex(pattern = keyword, ignore_case = TRUE), replacement = tgt, string = my)
print(my_rep)

[1] "base::sub() 함수이용 최초 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for

[1] "stringr::str_replace() 함수이용 최초 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for

In [24]:
# 최초 춯현내용 변경여부 파악

src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴

# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('원본패턴 %s 출현횟수:', src) 
gregexpr(pattern = src, text = my_sub, ignore.case = TRUE) %>% regmatches(x = my_sub) %>% lengths %>% print
str_count(regex(pattern = src, ignore_case = TRUE), string = my_sub) %>% print

sprintf('목표패턴 %s 출현횟수:', tgt)
gregexpr(pattern = tgt, text = my_rep, ignore.case = TRUE) %>% regmatches(x = my_rep) %>% lengths %>% print
str_count(regex(pattern = tgt, ignore_case = TRUE), string = my_rep) %>% print

[1] "원본패턴 Packages 출현횟수:"

[1] 6
[1] 6


[1] "목표패턴 library 출현횟수:"

[1] 1
[1] 1


### 모든 출현내용 변경

In [25]:
# 텍스트데이터내 원본패턴 중 모든 출현내용을 목표패턴으로 변경

sprintf('base::gsub() 함수이용 모든 출현내용 변경')
my_gsub <- gsub(pattern = src, replacement = tgt, x = my, ignore.case = TRUE)
print(my_gsub)

sprintf('stringr::str_replace_all() 함수이용 모든 출현내용 변경')
my_rep_all <- str_replace_all(regex(pattern = keyword, ignore_case = TRUE), replacement = tgt, string = my)
print(my_rep_all)

[1] "base::gsub() 함수이용 모든 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These library are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which library are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN library, and R-Forge, a central platform for th

[1] "stringr::str_replace_all() 함수이용 모든 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These library are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which library are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN library, and R-Forge, a central platform for th

In [26]:
# 모든 춯현내용 변경여부 파악

src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴

# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('원본패턴 %s 출현횟수:', src) 
gregexpr(pattern = src, text = my_gsub, ignore.case = TRUE) %>% regmatches(x = my_gsub) %>% lengths %>% print
str_count(regex(pattern = src, ignore_case = TRUE), string = my_gsub) %>% print

sprintf('목표패턴 %s 출현횟수:', tgt)
gregexpr(pattern = tgt, text = my_rep_all, ignore.case = TRUE) %>% regmatches(x = my_rep_all) %>% lengths %>% print
str_count(regex(pattern = tgt, ignore_case = TRUE), string = my_rep_all) %>% print

[1] "원본패턴 Packages 출현횟수:"

[1] 0
[1] 0


[1] "목표패턴 library 출현횟수:"

[1] 7
[1] 7


# <font color = 'blue'>텍스트셋에서 특정패턴 탐색: 대상객체가 여러 개 요소인 경우(p.59, p.73)</font>
* 원본텍스트셋 --> 문단단위 분해 --> 각 문단별 문장단위 분해 --> 순수 문장단위화
* base패키지(p.59)와 stringr패키지(p.73) 병행사용
* 텍스트셋 안에 특정패턴의 포함여부, 출현위치, 출현내용, 출현횟수 파악
* 텍스트셋 안에 원본패턴을 목표패턴으로 변경

## 텍스트 데이터 준비
* 원본 --> 문단단위(리스트구조) --> 문장단위(리스트구조) --> 순수문장단위(벡터구조)

### 원본 텍스트데이터

In [27]:
# 분석대상 텍스트 데이터 원본 준비
my <- 'The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. 
The "Task Views" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for the collaborative development of R packages, R-related software, and projects. R-Forge also hosts many unpublished beta packages, and development versions of CRAN packages.'

sprintf('원본내용:')
print(my)

sprintf('객체유형: %s객체', class(my))
sprintf('객체길이: %d요소', length(my))

sprintf('원본글자수: %d글자', nchar(my))
# - sprintf('검색대상 객체글자수: %d글자', str_length(my))

[1] "원본내용:"

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform fo

[1] "객체유형: character객체"

[1] "객체길이: 1요소"

[1] "원본글자수: 1166글자"

### 원본을 문단(paragraph) 단위 분해

In [28]:
# 원본을 문단(paragraph) 단위로 분해
my_para <- strsplit(my, split = '\n')
# - my_para <- stringr::str_split(my, pattern = '\n') 으로 코딩해도 됨

sprintf('객체내용:')
print(my_para)

sprintf('객체유형: %s객체', class(my_para))
sprintf('객체길이: %d요소', length(my_para))
sprintf('문단갯수: %d개', lengths(my_para))

sprintf('문단별 글자수:') 
nchar(my_para[[1]]) %>% print
# - stringr::str_length(my_para[[1]]) %>% print 으로 코딩해도 됨

[1] "객체내용:"

[[1]]
[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. "                                                                                                                                                                                                                                                   
[2] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which package

[1] "객체유형: list객체"

[1] "객체길이: 1요소"

[1] "문단갯수: 2개"

[1] "문단별 글자수:"

[1] 462 703


### 각 문단을 문장(sentence) 단위 분해

In [29]:
# 각 문단(paragraph)을 문장(sentence) 단위로 분해
my_para_sent <- strsplit(my_para[[1]], split = '\\. ')
# - my_para <- stringr::str_split(my_para, pattern = '\\.')으로 코딩해도 됨

sprintf('객체내용:')
print(my_para_sent)

sprintf('객체유형: %s객체', class(my_para_sent))
sprintf('문단길이: %d요소', length(my_para_sent))

sprintf('문단별 문장갯수:')
lengths(my_para_sent) %>% print

sprintf('문단별 글자갯수:')
nchar(my_para_sent) %>% print
# - str_length(my_para_sent) %>% print 으로 코딩해도 됨

sprintf('문단의 문장별 글자갯수:')
sapply(my_para_sent, nchar) %>% print
# - map(my_para_sent, str_length) %>% print 으로 코딩해도 됨

[1] "객체내용:"

[[1]]
[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                              
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                         

[[2]]
[1] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available"
[2] "R has also been identified by the FDA as suitable for interpreting data

[1] "객체유형: list객체"

[1] "문단길이: 2요소"

[1] "문단별 문장갯수:"

[1] 3 4


[1] "문단별 글자갯수:"

[1] 469 717


[1] "문단의 문장별 글자갯수:"

[[1]]
[1] 197  87 172

[[2]]
[1] 290  94 220  93



### 순수 문장(sentence) 단위화(벡터객체)

In [30]:
# 문단구조(리스트객체)를 해제하고 순수한 문장(sentence) 단위(벡터객체)로 구성
my_sent <- unlist(my_para_sent)

sprintf('객체내용:')
print(my_sent)

sprintf('객체유형: %s객체', class(my_sent))
sprintf('문단길이: %d요소', length(my_sent))
sprintf('문장갯수: %d개', length(my_sent)) %>% print

sprintf('문장별 글자갯수:')
nchar(my_sent) %>% print
# - str_length(my_para_sent) %>% print 으로 코딩해도 됨

[1] "객체내용:"

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                                                                                                                        
[4] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields 

[1] "객체유형: character객체"

[1] "문단길이: 7요소"

[1] "문장갯수: 7개"


[1] "문장별 글자갯수:"

[1] 197  87 172 290  94 220  93


## 텍스트셋에 특정패턴 포함여부 파악
* base:: 논리값으로 grep(), 요소인덱스번호로 grepl()
* stringr:: 논리값으로 str_detect(), 요소인덱스번호로 str_detect() & which()

In [31]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 포함여부를 논리값으로

In [32]:
# 텍스트셋 안에 특정패턴 포함여부를 논리값으로 도출 
# - 분석대상 텍스트데이터를 구성하는 여러 개 텍스트요소들 중에서 
#   검색패턴이 들어 있는 요소는 TRUE, 없는 요소는 FALSE 출력

sprintf('base::grepl() 함수이용 특정패턴 포함유무')
grepl(pattern = keyword, x = my_sent) # 대소문자 무시옵션: FALSE
grepl(pattern = keyword, x = my_sent, ignore.case = TRUE) # 대소문자 무시옵션: TRUE

sprintf('stringr::str_detect() 함수이용 특정패턴 포함유무')
str_detect(pattern = keyword, string = my_sent) %>% print # 대소문자 무시옵션: FALSE
str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% print # 대소문자 무시옵션: TRUE

[1] "base::grepl() 함수이용 특정패턴 포함유무"

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE

[1]  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE

[1] "stringr::str_detect() 함수이용 특정패턴 포함유무"

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[1]  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE


### 포함여부를 요소인덱스번호로

In [33]:
# 텍스트셋 안에 특정패턴 포함여부를 요소인덱스번호로 도출 
# - 분석대상 텍스트데이터를 구성하는 여러 개 텍스트요소들 중에서 
#   검색패턴이 들어 있는 요소인덱스 번호만을 출력함
# - 검색패턴이 없는 텍스트요소들은 아무것도 출력이 안됨
# - 만일 하나라도 해당하는 텍스트요소가 없으면, NULL의미를 가진 integer(0) 문구가 나옴

sprintf('base::grep() 함수이용 특정패턴 포함 인덱스')
grep(pattern = keyword, x = my_sent) %>% print # 대소문자 무시옵션: FALSE
grep(pattern = keyword, x = my_sent, ignore.case = TRUE) %>% print # 대소문자 무시옵션: TRUE

sprintf('stringr::str_detect() & base::which() 함수이용 특정패턴 포함 인덱스')
str_detect(pattern = keyword, string = my_sent) %>% which %>% print # 대소문자 무시옵션: FALSE
str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% which %>% print # 대소문자 무시옵션: TRUE

[1] "base::grep() 함수이용 특정패턴 포함 인덱스"

integer(0)
[1] 1 2 4 6 7


[1] "stringr::str_detect() & base::which() 함수이용 특정패턴 포함 인덱스"

integer(0)
[1] 1 2 4 6 7


### 포함여부를 요소내용으로

In [34]:
# 텍스트셋 안에 특정패턴 포함여부를 해당요소 내용으로 도출 
# - 분석대상 텍스트데이터를 구성하는 여러 개 텍스트요소들 중에서 
# - 검색패턴이 없는 텍스트요소들은 아무것도 출력이 안됨
# - 만일 하나라도 해당하는 텍스트요소가 없으면, NULL의미를 가진 character(0) 문구가 나옴

#### base::grep() 함수이용

In [35]:
sprintf('base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE')
print(grep(pattern = keyword, x = my_sent, value = TRUE))

cat('\n')

sprintf('base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE')
print(grep(pattern = keyword, x = my_sent, ignore.case = TRUE, value = TRUE))

[1] "base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE"

character(0)



[1] "base::grep() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE"

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available"
[4] "Other R package resources include Crantastic, a community site for rating and reviewing all CRAN p

#### stringr::str_detect() 함수이용

In [36]:
sprintf('stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE')
my_det <- str_detect(pattern = keyword, string = my_sent) 
my_det %>% print
my_sent[my_det] %>% print

cat('\n')

sprintf('stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE')
my_det <- str_detect(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) 
my_det %>% print
my_sent[my_det] %>% print

[1] "stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 FALSE"

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE
character(0)



[1] "stringr::str_detect() 함수이용 특정패턴 포함 요소내용: 대소문자 무시옵션 TRUE"

[1]  TRUE  TRUE FALSE  TRUE FALSE  TRUE  TRUE
[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available"
[4] "Other R package resources include Crantastic, a comm

## 텍스트셋에 특정패턴 출현위치 파악
* base:: 최초출현위치 regexpr(), 모든출현위치 gregexpr(), 중첩검색패턴 최초출현위치 regexec()
* stringr:: 최초출현위치 str_locate(), 모든출현위치 str_locate_all()

In [37]:
# 대상 텍스트셋 재확인
print(my_sent)

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                                                                                                                        
[4] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields 

In [38]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현위치

In [39]:
# 텍스트셋 안에 특정패턴 최초 출현위치 파악 
# - 분석대상 텍스트데이터를 구성하는 여러 개 텍스트요소들마다 
#   검색패턴이 출현하는 최초 위치(자리)를 파악해 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, stringr::str_locate()은 NA를 출력함

#### base::regexpr() 함수이용

In [40]:
sprintf('base::regexpr() 함수이용 최초 출현위치: 대소문자 무시옵션 FALSE')
regexpr(pattern = keyword, text = my_sent) %>% print

cat('\n')

sprintf('base::regexpr() 함수이용 최초 출현위치: 대소문자 무시옵션 TRUE')
regexpr(pattern = keyword, text = my_sent, ignore.case = TRUE) %>% print

[1] "base::regexpr() 함수이용 최초 출현위치: 대소문자 무시옵션 FALSE"

[1] -1 -1 -1 -1 -1 -1 -1
attr(,"match.length")
[1] -1 -1 -1 -1 -1 -1 -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



[1] "base::regexpr() 함수이용 최초 출현위치: 대소문자 무시옵션 TRUE"

[1]  57   7  -1 269  -1  98  42
attr(,"match.length")
[1]  8  8 -1  8 -1  8  8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE


#### stringr::str_locate() 함수이용

In [41]:
sprintf('stringr::str_locate() 함수이용 최초 출현위치: 대소문자 무시옵션 FALSE')
str_locate(pattern = keyword, string = my_sent)  %>% print

cat('\n')

sprintf('stringr::str_locate() 함수이용 최초 출현위치: 대소문자 무시옵션 TRUE')
str_locate(regex(pattern = keyword, ignore_case = TRUE), string = my_sent)  %>% print

[1] "stringr::str_locate() 함수이용 최초 출현위치: 대소문자 무시옵션 FALSE"

     start end
[1,]    NA  NA
[2,]    NA  NA
[3,]    NA  NA
[4,]    NA  NA
[5,]    NA  NA
[6,]    NA  NA
[7,]    NA  NA



[1] "stringr::str_locate() 함수이용 최초 출현위치: 대소문자 무시옵션 TRUE"

     start end
[1,]    57  64
[2,]     7  14
[3,]    NA  NA
[4,]   269 276
[5,]    NA  NA
[6,]    98 105
[7,]    42  49


### 모든 출현위치

In [42]:
# 텍스트셋 안에 특정패턴 모든 출현위치 파악 
# - 분석대상 텍스트데이터를 구성하는 여러 개 텍스트요소들마다 
#   검색패턴이 출현하는 모든 위치(자리)를 파악해 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::gregexpr()은 -1을, stringr::str_locate_all()은 NA를 출력함

#### base::gregexpr() 함수이용

In [43]:
sprintf('base::gregexpr() 함수이용 모든 출현위치: 대소문자 무시옵션 FALSE')
gregexpr(pattern = keyword, text = my_sent) %>% print

[1] "base::gregexpr() 함수이용 모든 출현위치: 대소문자 무시옵션 FALSE"

[[1]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[2]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[3]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[4]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[5]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[6]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[7]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



In [44]:
sprintf('base::gregexpr() 함수이용 모든 출현위치: 대소문자 무시옵션 TRUE')
gregexpr(pattern = keyword, text = my_sent, ignore.case = TRUE) %>% print

[1] "base::gregexpr() 함수이용 모든 출현위치: 대소문자 무시옵션 TRUE"

[[1]]
[1] 57
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[2]]
[1] 7
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[3]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[4]]
[1] 269
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[5]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[6]]
[1]  98 179
attr(,"match.length")
[1] 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[7]]
[1] 42 85
attr(,"match.length")
[1] 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



#### stringr::str_locate_all() 함수이용 

In [45]:
sprintf('stringr::str_locate_all() 함수이용 모든 출현위치: 대소문자 무시옵션 FALSE')
str_locate_all(pattern = keyword, string = my_sent) %>% print

cat('\n#--------------------\n\n')

sprintf('stringr::str_locate_all() 함수이용 모든 출현위치: 대소문자 무시옵션 TRUE')
str_locate_all(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% print

[1] "stringr::str_locate_all() 함수이용 모든 출현위치: 대소문자 무시옵션 FALSE"

[[1]]
     start end

[[2]]
     start end

[[3]]
     start end

[[4]]
     start end

[[5]]
     start end

[[6]]
     start end

[[7]]
     start end


#--------------------



[1] "stringr::str_locate_all() 함수이용 모든 출현위치: 대소문자 무시옵션 TRUE"

[[1]]
     start end
[1,]    57  64

[[2]]
     start end
[1,]     7  14

[[3]]
     start end

[[4]]
     start end
[1,]   269 276

[[5]]
     start end

[[6]]
     start end
[1,]    98 105
[2,]   179 186

[[7]]
     start end
[1,]    42  49
[2,]    85  92



## 텍스트셋에 특정패턴 출현내용 파악
* base:: 최초출현내용 regexpr() & regmatches(), 모든출현내용 regexpr() & regmatches()  
* stringr:: 최초출현내용 str_extract(), 모든출현내용 str_extract_all()

In [46]:
# 대상 텍스트셋 재확인
print(my_sent)

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                                                                                                                        
[4] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields 

In [47]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현내용: 세부연산과정 상세
* base::regexpr() & regmatches 함수이용 최초 출현내용: 세부연산과정 상세

In [48]:
# 텍스트셋 안에 특정패턴 최초 출현내용 파악 
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, regmatches는 character(0)을 출력 
# - 검색패턴이 해당 텍스트요소에 들어 있으면 
#   base::gregexpr()은 최초 출현위치 인덱스를, regmatches는 최초 출현내용을 출력 

In [49]:
# 검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우
sprintf('검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우')

tmp_loc <- regexpr(pattern = keyword, text = my_sent, ignore.case = FALSE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my_sent, m = tmp_loc)
print(tmp_ptr)

cat('\n#--------------------\n\n')

sprintf('검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우')
tmp_loc <- regexpr(pattern = keyword, text = my_sent, ignore.case = TRUE)
print(tmp_loc)
cat('\n')
tmp_ptr <- regmatches(x = my_sent, m = tmp_loc)
print(tmp_ptr)

[1] "검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우"

[1] -1 -1 -1 -1 -1 -1 -1
attr(,"match.length")
[1] -1 -1 -1 -1 -1 -1 -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

character(0)

#--------------------



[1] "검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우"

[1]  57   7  -1 269  -1  98  42
attr(,"match.length")
[1]  8  8 -1  8 -1  8  8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[1] "packages" "packages" "packages" "packages" "packages"


### 최초 출현내용: 파이프연산자 이용
* base::regexpr() & regmatches 함수이용 최초 출현내용: 파이프연산자 이용
* 'stringr::str_extract() 함수이용 최초 출현내용

In [50]:
# 텍스트셋 안에 특정패턴 최초 출현내용 파악
# - 현재 1개 텍스트요소에 대한 검색이므로 검색패턴이 출현하는 모든 내용도 1개만 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::regexpr()은 -1을, regmatches는 character(0), 
#   stringr::str_extract()은 NA를 출력함

#### base::regexpr() & regmatches  함수이용

In [51]:
sprintf('검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우')
regexpr(pattern = keyword, text = my_sent) %>% regmatches(x = my_sent) %>% print

sprintf('검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우')
regexpr(pattern = keyword, text = my_sent, ignore.case = TRUE) %>% regmatches(x = my_sent) %>% print

[1] "검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우"

character(0)


[1] "검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우"

[1] "packages" "packages" "packages" "packages" "packages"


#### stringr::str_extract() 함수이용

In [52]:
sprintf('검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우')
str_extract(pattern = keyword, string = my_sent) %>% print

sprintf('검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우')
str_extract(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% print

[1] "검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우"

[1] NA NA NA NA NA NA NA


[1] "검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우"

[1] "packages" "packages" NA         "packages" NA         "packages" "packages"


### 모든 출현내용: 세부연산과정 상세
* base::gregexpr() & regmatches 함수이용 모든 출현내용: 세부연산과정 상세

In [53]:
# 텍스트셋 안에 특정패턴 모든 출현내용 파악 
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 
#   base::gregexpr()은 -1을, regmatches는 character(0)을 출력 
# - 검색패턴이 해당 텍스트요소에 들어 있으면 
#   base::gregexpr()은 모든 출현위치 인덱스를, regmatches는 모든 출현내용을 출력 

In [54]:
sprintf('검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우')
tmp_loc <- gregexpr(pattern = keyword, text = my_sent, ignore.case = FALSE)
print(tmp_loc)

[1] "검색패턴 대소문자 구별옵션 ignore.case = FALSE인 경우"

[[1]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[2]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[3]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[4]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[5]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[6]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[7]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



In [55]:
sprintf('검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우')
tmp_ptr <- regmatches(x = my_sent, m = tmp_loc)
print(tmp_ptr)

[1] "검색패턴 대소문자 구별옵션 ignore.case = TRUE인 경우"

[[1]]
character(0)

[[2]]
character(0)

[[3]]
character(0)

[[4]]
character(0)

[[5]]
character(0)

[[6]]
character(0)

[[7]]
character(0)



### 모든 출현내용: 파이프연산자 이용
* base::gregexpr() & regmatches 함수이용 모든 출현내용: 파이프연산자 이용
* stringr::str_extract_all() 함수이용 모든 출현내용

In [56]:
# 텍스트셋 안에 특정패턴 모든 출현내용 파악
# - 현재 1개 텍스트요소에 대한 검색인데, 검색패턴이 출현하는 모든 내용을 출력함
# - 검색패턴이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함

#### base::gregexpr() & regmatches 함수이용

In [57]:
sprintf('대소문자 무시옵션: FALSE')
gregexpr(pattern = keyword, text = my_sent) %>% regmatches(x = my_sent) %>% print

[1] "대소문자 무시옵션: FALSE"

[[1]]
character(0)

[[2]]
character(0)

[[3]]
character(0)

[[4]]
character(0)

[[5]]
character(0)

[[6]]
character(0)

[[7]]
character(0)



In [58]:
sprintf('대소문자 무시옵션: TRUE')
gregexpr(pattern = keyword, text = my_sent, ignore.case = TRUE) %>% regmatches(x = my_sent) %>% print

[1] "대소문자 무시옵션: TRUE"

[[1]]
[1] "packages"

[[2]]
[1] "packages"

[[3]]
character(0)

[[4]]
[1] "packages"

[[5]]
character(0)

[[6]]
[1] "packages" "packages"

[[7]]
[1] "packages" "packages"



#### stringr::str_extract_all() 함수이용

In [59]:
sprintf('대소문자 무시옵션: FALSE')
str_extract_all(pattern = keyword, string = my_sent) %>% print

[1] "대소문자 무시옵션: FALSE"

[[1]]
character(0)

[[2]]
character(0)

[[3]]
character(0)

[[4]]
character(0)

[[5]]
character(0)

[[6]]
character(0)

[[7]]
character(0)



In [60]:
sprintf('대소문자 무시옵션: TRUE')
str_extract_all(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% print

[1] "대소문자 무시옵션: TRUE"

[[1]]
[1] "packages"

[[2]]
[1] "packages"

[[3]]
character(0)

[[4]]
[1] "packages"

[[5]]
character(0)

[[6]]
[1] "packages" "packages"

[[7]]
[1] "packages" "packages"



## 테스트셋에 특정패턴 출현횟수 파악
* base:: gregexpr() & & regmatches & lenghts
* stringr:: str_count()

In [61]:
# 대상 텍스트셋 재확인
print(my_sent)

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                                                                                                                        
[4] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields 

In [62]:
# 검색용 키워드 준비
keyword = 'Packages'
# - 검색패턴에 대소문자가 혼용되어 있는 상태임

### 출현횟수: 세부연산과정 상세
* base::gregexpr() & regmatches & lengths 함수이용 출현횟수: 세부연산과정 상세

#### 해당표현이 출현한 모든 위치를 파악

In [63]:
tmp_loc <- gregexpr(pattern = keyword, text = my_sent, ignore.case = TRUE)
print(tmp_loc)

[[1]]
[1] 57
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[2]]
[1] 7
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[3]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[4]]
[1] 269
attr(,"match.length")
[1] 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[5]]
[1] -1
attr(,"match.length")
[1] -1
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[6]]
[1]  98 179
attr(,"match.length")
[1] 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE

[[7]]
[1] 42 85
attr(,"match.length")
[1] 8 8
attr(,"index.type")
[1] "chars"
attr(,"useBytes")
[1] TRUE



#### 해당표현이 출현한 모든 내용을 파악: 리스트객체임

In [64]:
tmp_ptr <- regmatches(x = my_sent, m = tmp_loc)
print(tmp_ptr)

[[1]]
[1] "packages"

[[2]]
[1] "packages"

[[3]]
character(0)

[[4]]
[1] "packages"

[[5]]
character(0)

[[6]]
[1] "packages" "packages"

[[7]]
[1] "packages" "packages"



#### 대상텍스트에 변경대상 키워드 출현횟수를 파악

In [65]:
lengths(tmp_ptr) %>% print
# - 리스트객체에 대한 리스트항목별 세부요소갯수 산정

[1] 1 1 0 1 0 2 2


### 출현횟수: 파이프연산자 이용
* base::gregexpr() & regmatches  함수이용 출현횟수: 파이프연산자 이용
* stringr::str_count() 함수이용 출현횟수

In [66]:
# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('base::gregexpr() & regmatches  함수이용 출현횟수: 대소문자 무시옵션 FALSE')
gregexpr(pattern = keyword, text = my_sent) %>% regmatches(x = my_sent) %>% lengths %>% print

sprintf('base::gregexpr() & regmatches  함수이용 출현횟수: 대소문자 무시옵션 TRUE')
gregexpr(pattern = keyword, text = my_sent, ignore.case = TRUE) %>% regmatches(x = my_sent) %>% lengths %>% print

cat('\n#--------------------\n\n')

sprintf('stringr::str_count() 함수이용 출현횟수: 대소문자 무시옵션 FALSE')
str_count(pattern = keyword, string = my_sent) %>% print

sprintf('stringr::str_count() 함수이용 출현횟수: 대소문자 무시옵션 TRUE')
str_count(regex(pattern = keyword, ignore_case = TRUE), string = my_sent) %>% print

[1] "base::gregexpr() & regmatches  함수이용 출현횟수: 대소문자 무시옵션 FALSE"

[1] 0 0 0 0 0 0 0


[1] "base::gregexpr() & regmatches  함수이용 출현횟수: 대소문자 무시옵션 TRUE"

[1] 1 1 0 1 0 2 2

#--------------------



[1] "stringr::str_count() 함수이용 출현횟수: 대소문자 무시옵션 FALSE"

[1] 0 0 0 0 0 0 0


[1] "stringr::str_count() 함수이용 출현횟수: 대소문자 무시옵션 TRUE"

[1] 1 1 0 1 0 2 2


## 원본패턴을 목표패턴으로 변경
* base:: 최초패턴을 변경 sub(), 모든패턴을 변경 gsub()
* stringr:: 최초패턴을 변경 str_replace(), 모든패턴을 변경 str_replace_all()

In [67]:
# 대상 텍스트셋 재확인
print(my_sent)

[1] "The capabilities of R are extended through user-created packages, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc"                                                                                               
[2] "These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran"                                                                                                                                                                                                             
[3] "The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving"                                                                                                                        
[4] "The \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields 

In [68]:
# 변경용 키워드 준비
src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴
# - 검색용 원본패턴에 대소문자가 혼용되어 있는 상태임

### 최초 출현내용 변경

In [69]:
# 텍스트데이터내 원본패턴 중 최초 출현내용을 목표패턴으로 변경

sprintf('base::sub() 함수이용 최초 출현내용 변경')
my_sub <- sub(pattern = src, replacement = tgt, x = my, ignore.case = TRUE)
print(my_sub)

sprintf('stringr::str_replace() 함수이용 최초 출현내용 변경')
my_rep <- str_replace(regex(pattern = keyword, ignore_case = TRUE), replacement = tgt, string = my)
print(my_rep)

[1] "base::sub() 함수이용 최초 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for

[1] "stringr::str_replace() 함수이용 최초 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These packages are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which packages are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN packages, and R-Forge, a central platform for

In [70]:
# 최초 춯현내용 변경여부 파악

src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴

# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('원본패턴 %s 출현횟수:', src) 
gregexpr(pattern = src, text = my_sub, ignore.case = TRUE) %>% regmatches(x = my_sub) %>% lengths %>% print
str_count(regex(pattern = src, ignore_case = TRUE), string = my_sub) %>% print

sprintf('목표패턴 %s 출현횟수:', tgt)
gregexpr(pattern = tgt, text = my_rep, ignore.case = TRUE) %>% regmatches(x = my_rep) %>% lengths %>% print
str_count(regex(pattern = tgt, ignore_case = TRUE), string = my_rep) %>% print

[1] "원본패턴 Packages 출현횟수:"

[1] 6
[1] 6


[1] "목표패턴 library 출현횟수:"

[1] 1
[1] 1


### 모든 출현내용 변경

In [71]:
# 텍스트데이터내 원본패턴 중 모든 출현내용을 목표패턴으로 변경

sprintf('base::gsub() 함수이용 모든 출현내용 변경')
my_gsub <- gsub(pattern = src, replacement = tgt, x = my, ignore.case = TRUE)
print(my_gsub)

sprintf('stringr::str_replace_all() 함수이용 모든 출현내용 변경')
my_rep_all <- str_replace_all(regex(pattern = keyword, ignore_case = TRUE), replacement = tgt, string = my)
print(my_rep_all)

[1] "base::gsub() 함수이용 모든 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These library are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which library are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN library, and R-Forge, a central platform for th

[1] "stringr::str_replace_all() 함수이용 모든 출현내용 변경"

[1] "The capabilities of R are extended through user-created library, which allow specialised statistical techniques, graphical devices, import/export capabilities, reporting tools (knitr, Sweave), etc. These library are developed primarily in R, and sometimes in Java, C, C++, and Fortran. The R packaging system is also used by researchers to create compendia to organise research data, code and report files in a systematic way for sharing and public archiving. \nThe \"Task Views\" page (subject list) on the CRAN website lists a wide range of tasks (in fields such as Finance, Genetics, High Performance Computing, Machine Learning, Medical Imaging, Social Sciences and Spatial Statistics) to which R has been applied and for which library are available. R has also been identified by the FDA as suitable for interpreting data from clinical research. Other R package resources include Crantastic, a community site for rating and reviewing all CRAN library, and R-Forge, a central platform for th

In [72]:
# 모든 춯현내용 변경여부 파악

src = 'Packages' # 원본패턴
tgt = 'library' # 목표패턴

# 대상텍스트에 변경대상 키워드 포함갯수를 파악
sprintf('원본패턴 %s 출현횟수:', src) 
gregexpr(pattern = src, text = my_gsub, ignore.case = TRUE) %>% regmatches(x = my_gsub) %>% lengths %>% print
str_count(regex(pattern = src, ignore_case = TRUE), string = my_gsub) %>% print

sprintf('목표패턴 %s 출현횟수:', tgt)
gregexpr(pattern = tgt, text = my_rep_all, ignore.case = TRUE) %>% regmatches(x = my_rep_all) %>% lengths %>% print
str_count(regex(pattern = tgt, ignore_case = TRUE), string = my_rep_all) %>% print

[1] "원본패턴 Packages 출현횟수:"

[1] 0
[1] 0


[1] "목표패턴 library 출현횟수:"

[1] 7
[1] 7


# End of Source